# Normalizing and Creating a General Data Set
- Consumption we need to normalize somehow to MWh
    - [We can find energy conversions here](https://www.eia.gov/tools/faqs/faq.php?id=667&t=6)
    - The end result in `total_consumption` will be in MWh
    - This meets my expectations as I expect in a month millions of MWh to power an entire state
- Balance is actually pretty clean, we just have to aggregate somehow to the daily
- Weather is also relatively clean, I left some things the same depending on the downstream processing

In [1]:
import pandas as pd
import altair as alt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

pd.set_option('display.max_columns', 500)
pd.set_option('future.no_silent_downcasting', True)
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

## Data Transforms

In [2]:
start_date = pd.to_datetime('2016-01-01')
coal = 0.88 * 2000 # kWh/pound * M/1000k * 2000pound/1ton * 1000thousand
nat_gas = 0.13 # kWh/cubic foot * M/1000k * 1000thousand
petrol_l = 12.90 / 42 # kWh/gallon * M/1000k * barrel/42gallon * 1000thousand
petrol_c = 1.18 * 2000 # kWh/pound * M/1000k * 2000pound/1ton * 1000thousand

In [3]:
balance = pd.read_csv("../../data/balance_sheet.csv", parse_dates=['data_date']) # megawatts
bcols = [
    'demand',
    'net_generation', 
    'total_interchange',	
    'net_generation_coal', 
    'net_generation_natural_gas', 
    'net_generation_nuclear',
    'net_generation_hydropower_and_pumped_storage', 
    'net_generation_solar', 
    'net_generation_wind', 
    'net_generation_other_fuel_sources'
]
balance[bcols] = balance[bcols].replace(',', '', regex=True).astype(int)
# balance.groupby('data_date')[bcols].mean().round(0).reset_index()
# balance.groupby([balance.data_date.dt.month, balance.data_date.dt.year])[bcols].mean().round(0)
# balance.groupby([balance.data_date.dt.day, balance.data_date.dt.month, balance.data_date.dt.year])[bcols].mean().round(0)

In [5]:
consumption = pd.read_csv("../../data/consumption_mi.csv", header=4).dropna(axis=0).drop(['units', 'source key'], axis=1)
consumption['description'] = consumption['description'].apply(lambda row: '_'.join(row.split(': ')[1].split(' ')))
consumption = consumption.T
consumption.columns = consumption.iloc[0, :]
consumption = consumption.drop('description', axis=0).reset_index(names='date')
consumption['date'] = pd.to_datetime(consumption['date'], format='%b %Y')
consumption['petroleum_liquids'] = consumption['petroleum_liquids'].replace('NM', 0.0)
consumption['total_consumption'] = consumption['coal'].astype(float)*coal + \
    consumption['petroleum_liquids'].astype(float)*petrol_l + \
    consumption['petroleum_coke'].astype(float)*petrol_c + \
    consumption['natural_gas'].astype(float)*nat_gas
consumption[consumption['date'] >= start_date].to_csv("../../data/targets.csv", index=False)

In [6]:
weather = pd.read_csv("../../data/WeatherReport.csv", parse_dates=['DATE'])
wcols = [
    'DAPR',
    'MDPR',
    'PRCP',
    'SN52',
    'SN53',
    'SNOW',
    'SNWD',
    'SX52',
    'SX53',
    'TMAX',
    'TMIN',
    'TOBS',
    'WESD',
    'WSFG',
    'WT01',
    'WT03',
    'WT04',
    'WT05',
    'WT06',
    'WT11'
]
# weather.groupby([weather.DATE.dt.month, weather.DATE.dt.year])[wcols].mean()
# weather[wcols].describe()
# weather[weather['DATE'] >= date].groupby([weather.DATE.dt.day, weather.DATE.dt.month, weather.DATE.dt.year])[wcols].mean()

In [7]:
b = balance.groupby([balance.data_date.dt.day, balance.data_date.dt.month, balance.data_date.dt.year])[bcols] \
    .mean() \
    .round(0)
b.index = b.index.set_names(['day', 'month', 'year'])
b = b.reset_index()
b.insert(1, 'date', pd.to_datetime(b[['day', 'month', 'year']]))
b = b.drop(['day', 'month', 'year'], axis=1)

In [9]:
w = weather[weather['DATE'] >= start_date].groupby([weather.DATE.dt.day, weather.DATE.dt.month, weather.DATE.dt.year])[wcols] \
    .mean() \
    .round(0)
w.index = w.index.set_names(['day', 'month', 'year'])
w = w.reset_index()
w.insert(1, 'date', pd.to_datetime(w[['day', 'month', 'year']]))
w = w.drop(['day', 'month', 'year'], axis=1)

In [10]:
b.merge(w, on='date').sort_values('date').to_csv("../../data/features.csv", index=False)

1. Blow up our data:
- Sum demand and get percents by day
    - If 110 MWh generated and on day 1 11 MWh was generate that is 10%
    - We say that 10% of energy was consumed
    - We are assuming that 1:1 consumption:generation on the day
2. Articulate the model change/play with FTN

## Let's Blow Our Targets

In [6]:
features = pd.read_csv("../../data/features.csv", parse_dates=['date'])
targets = pd.read_csv("../../data/targets.csv", parse_dates=['date'])

In [34]:
targets

,date,coal,petroleum_liquids,petroleum_coke,natural_gas,total_consumption
0,2016-01-01,2440.0,23.0,6.0,15120.0,4.310533e+06
1,2016-02-01,1966.0,17.0,25.0,15400.0,3.521167e+06
2,2016-03-01,1659.0,19.0,28.0,17555.0,2.988208e+06
3,2016-04-01,1331.0,17.0,35.0,21255.0,2.427928e+06
4,2016-05-01,1668.0,38.0,31.0,21153.0,3.011602e+06
...,...,...,...,...,...,...
97,2024-02-01,961.0,14.0,65.0,33998.0,1.849184e+06
98,2024-03-01,565.0,22.0,23.0,30692.0,1.052677e+06
99,2024-04-01,919.0,12.0,53.0,32405.0,1.746736e+06
100,2024-05-01,917.0,18.0,58.0,36500.0,1.755551e+06


In [24]:
totals = features.groupby([features.date.dt.month, features.date.dt.year])['demand'].sum()
totals.index = totals.index.set_names(['month', 'year'])
totals = totals.reset_index()
totals.insert(1, 'date', pd.to_datetime(totals[['month', 'year']].assign(day=1)))
totals = totals.drop(['month', 'year'], axis=1)
totals.columns = ['date', 'total_demand']

In [35]:
percents = totals.merge(features[['date', 'demand']], on='date', how='right')
percents['total_demand'] = percents['total_demand'].ffill()
percents['percents'] = percents['demand'] / percents['total_demand']

In [55]:
new_targets = targets.merge(percents[['date', 'percents']], on='date', how='right')
new_targets['total_consumption'] = new_targets['total_consumption'].ffill()
new_targets = new_targets.ffill()
new_targets['consumption'] = (new_targets['total_consumption'] * new_targets['percents']).round(0)
new_targets = new_targets.drop(['percents', 'total_consumption'], axis=1)
new_targets

,date,coal,petroleum_liquids,petroleum_coke,natural_gas,consumption
0,2016-01-01,2440.0,23.0,6.0,15120.0,127103.0
1,2016-01-02,2440.0,23.0,6.0,15120.0,127530.0
2,2016-01-03,2440.0,23.0,6.0,15120.0,127693.0
3,2016-01-04,2440.0,23.0,6.0,15120.0,143012.0
4,2016-01-05,2440.0,23.0,6.0,15120.0,147147.0
...,...,...,...,...,...,...
3179,2024-09-14,1367.0,0.0,73.0,34578.0,145072.0
3180,2024-09-15,1367.0,0.0,73.0,34578.0,145998.0
3181,2024-09-16,1367.0,0.0,73.0,34578.0,155157.0
3182,2024-09-17,1367.0,0.0,73.0,34578.0,154206.0


In [59]:
features.merge(new_targets[['date', 'consumption']], on='date').to_csv("../../data/dataset.csv", index=False)

In [60]:
pd.read_csv("../../data/dataset.csv")

,date,demand,net_generation,total_interchange,net_generation_coal,net_generation_natural_gas,net_generation_nuclear,net_generation_hydropower_and_pumped_storage,net_generation_solar,net_generation_wind,net_generation_other_fuel_sources,DAPR,MDPR,PRCP,SN52,SN53,SNOW,SNWD,SX52,SX53,TMAX,TMIN,TOBS,WESD,WSFG,WT01,WT03,WT04,WT05,WT06,WT11,consumption
0,2016-01-01,69621.0,67425.0,-3254.0,39962.0,20101.0,11610.0,746.0,0.0,3865.0,1122.0,NaN,NaN,2.0,6.0,17.0,5.0,96.0,17.0,17.0,-27.0,-77.0,-56.0,38.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,127103.0
1,2016-01-02,69855.0,67906.0,-2904.0,39962.0,20101.0,11610.0,746.0,0.0,3865.0,1122.0,NaN,NaN,0.0,6.0,17.0,1.0,92.0,12.0,17.0,-17.0,-78.0,-53.0,38.0,NaN,1.0,NaN,NaN,NaN,1.0,NaN,127530.0
2,2016-01-03,69944.0,67305.0,-3588.0,39962.0,20101.0,11610.0,746.0,0.0,3865.0,1122.0,NaN,NaN,1.0,6.0,17.0,3.0,85.0,12.0,17.0,3.0,-69.0,-35.0,38.0,192.0,1.0,NaN,1.0,NaN,1.0,1.0,127693.0
3,2016-01-04,78335.0,77001.0,-2642.0,39962.0,20101.0,11610.0,746.0,0.0,3865.0,1122.0,NaN,NaN,8.0,3.0,17.0,17.0,92.0,12.0,17.0,-21.0,-84.0,-75.0,51.0,192.0,1.0,NaN,1.0,NaN,1.0,1.0,143012.0
4,2016-01-05,80600.0,81083.0,-896.0,39962.0,20101.0,11610.0,746.0,0.0,3865.0,1122.0,NaN,NaN,0.0,-11.0,17.0,1.0,90.0,8.0,17.0,-30.0,-126.0,-86.0,51.0,192.0,1.0,NaN,1.0,NaN,1.0,1.0,147147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3179,2024-09-14,75856.0,72728.0,-4453.0,21114.0,31613.0,11557.0,1049.0,1925.0,5164.0,306.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,282.0,136.0,169.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145072.0
3180,2024-09-15,76340.0,72659.0,-5173.0,21023.0,29004.0,11520.0,1053.0,1917.0,7857.0,283.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,284.0,144.0,176.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145998.0
3181,2024-09-16,81129.0,78197.0,-4749.0,22613.0,31772.0,11471.0,1118.0,2015.0,8937.0,272.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,295.0,141.0,163.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155157.0
3182,2024-09-17,80632.0,79791.0,-2634.0,22567.0,30658.0,11452.0,995.0,2400.0,11449.0,272.0,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,294.0,130.0,155.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154206.0
